<a href="https://colab.research.google.com/github/rabiayilmazz/Android/blob/main/steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
# Kütüpaneler

import cv2
import numpy as np
import types
from google.colab.patches import cv2_imshow 
 

In [33]:
# drive bağlantısı

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# format değişimi

def messageToBinary(message):
  if type(message) == str:
    return ''.join([ format(ord(i), "08b") for i in message ])
  
  elif type(message) == bytes or type(message) == np.ndarray:
    return [ format(i, "08b") for i in message ]
  
  elif type(message) == int or type(message) == np.uint8:
    return format(message, "08b")
  
  else:
    raise TypeError("Input type not supported")

In [35]:
messageToBinary("rabia")

'0111001001100001011000100110100101100001'

In [36]:
messageToBinary(str.encode("rabia"))

['01110010', '01100001', '01100010', '01101001', '01100001']

In [37]:
messageToBinary("###")

'001000110010001100100011'

In [38]:
# Alınan mesajı resme gizler ve resmin pixellerini döndürür

def hideData(image, secret_message):

  n_bytes = image.shape[0] * image.shape[1] * 3 // 8
  print("Şİfrelenebilecek maximum byte:", n_bytes)

  # girilen metnin boyutu resmin boyutundan büyük olmamalı
  if len(secret_message) > n_bytes:
      raise ValueError("Error encountered insufficient bytes, need bigger image or less data !!")
  
  secret_message += "#####" # sonlandırma için 
  print("secret_message:  "+secret_message)
  data_index = 0

  # 10110101100101
  binary_secret_msg = messageToBinary(secret_message)

  data_len = len(binary_secret_msg)

  for values in image:
      for pixel in values:
          # rgb den binarye
          r, g, b = messageToBinary(pixel)

          if data_index < data_len:
              print("binary_secret_msg[data_index] " + binary_secret_msg[data_index])
              print("r " + r)
              print("r[:-1] + binary_secret_msg[data_index] " + r[:-1] + binary_secret_msg[data_index])
              pixel[0] = int(r[:-1] + binary_secret_msg[data_index], 2)
              data_index += 1


          if data_index < data_len:
              pixel[1] = int(g[:-1] + binary_secret_msg[data_index], 2)
              data_index += 1

          if data_index < data_len:
              pixel[2] = int(b[:-1] + binary_secret_msg[data_index], 2)
              data_index += 1

          if data_index >= data_len:
              break

  return image

In [39]:
def showData(image):

  binary_data = ""
  for values in image:
      for pixel in values:
          r, g, b = messageToBinary(pixel) #resime yerleştirilip pixeli değiştirilen renklerin değerleri tekrar binarye çevirme
          binary_data += r[-1] #son değere gizlenen binaryi okuyup binary_data a yazma
          binary_data += g[-1] 
          binary_data += b[-1] 
  # 8 er şekilde ayırmak içn
  all_bytes = [ binary_data[i: i+8] for i in range(0, len(binary_data), 8) ]
  # convert from bits to characters
  decoded_data = ""
  for byte in all_bytes:
      decoded_data += chr(int(byte, 2))
      print("decoded_data " + decoded_data)
      if decoded_data[-5:] == "#####": # "#####" i görünce sonlansın 
          break
  #print(decoded_data) # hastaglı deşifrelenemiş veri
  return decoded_data[:-5] # hastagsız asıl veri

In [40]:
def encode_text(): 
  image_name = input("Resim adı girini (örnek: a.png): ") 
  image = cv2.imread(image_name) # resim nesnesine dönüştürüp pixel ve shape okumak için
  
  print("Resmin shape değeri: ",image.shape) # alınan resmin x ve y eksenindeki uzunluğu
  print("Tekrar boyutlanmış - içerisine veri gizlenecek olan resim: ")
  resized_image = cv2.resize(image, (30, 30)) # tekrar boyutlandırmak için - saklanacakverinin boyutuna etki eder
  cv2_imshow(resized_image) # resim görüntüleme
  
      
  data = input("Resme gizlenecek veriyi giriniz : ") 
  if (len(data) == 0): 
    raise ValueError('HATA! Boş geçilemez. Lütfen veri giriniz')
  
  filename = input("Metin gizlenmiş resmin adını giriniz (örnek: gizli_!_test.png): ")
  encoded_image = hideData(image, data) # veri gizleme fonksiyonuna alınan metni ve resmi gönderip resme veriyi gizlemek için yazdığım fonksiyon
  cv2.imwrite(filename, encoded_image) # verinin gizlendiği resmin ekrana basılması için

 

In [41]:
def decode_text():
  image_name = input("Veri saklanan resmin adını giriniz (örnek: gizli_!_test.png) :") 
  image = cv2.imread(image_name) # resmi işlenebilir hale getirebilmek için

  print("Resme gizlenen metin: ")
  resized_image = cv2.resize(image, (30,30))  
  cv2_imshow(resized_image) 
    
  text = showData(image) # gizlenen metni deşifrelemek için
  return text


In [42]:
def Steganography(): 
    a = input(" Steganography \n 1. Veri sakla \n 2. Saklananveriyi oku \n Yukarıdaki seçeneklerden birini seçiniz: ")
    userinput = int(a)
    if (userinput == 1):
      print("\nVeri gizleniyor....")
      encode_text() 
          
    elif (userinput == 2):
      print("\nVeri deşifreleniyor....") 
      print("Deşifrelenmiş verinin sonucu " + decode_text()) 
    else: 
        raise Exception("Lütfen yukarıdaki gibi geçerli ifade giriniz!") 

In [43]:
Steganography() 

 Steganography 
 1. Veri sakla 
 2. Saklananveriyi oku 
 Yukarıdaki seçeneklerden birini seçiniz: 1

Veri gizleniyor....
Resim adı girini (örnek: a.png): a.png
Resmin shape değeri:  (353, 572, 3)
Tekrar boyutlanmış - içerisine veri gizlenecek olan resim: 


Resme gizlenecek veriyi giriniz : selam naber
Metin gizlenmiş resmin adını giriniz (örnek: gizli_!_test.png): gizli_!_test.png
Şİfrelenebilecek maximum byte: 75718
secret_message:  selam naber#####
binary_secret_msg[data_index] 0
r 00011110
r[:-1] + binary_secret_msg[data_index] 00011110
binary_secret_msg[data_index] 1
r 00011110
r[:-1] + binary_secret_msg[data_index] 00011111
binary_secret_msg[data_index] 1
r 11110001
r[:-1] + binary_secret_msg[data_index] 11110001
binary_secret_msg[data_index] 1
r 00011110
r[:-1] + binary_secret_msg[data_index] 00011111
binary_secret_msg[data_index] 0
r 11011011
r[:-1] + binary_secret_msg[data_index] 11011010
binary_secret_msg[data_index] 1
r 10000011
r[:-1] + binary_secret_msg[data_index] 10000011
binary_secret_msg[data_index] 1
r 00011110
r[:-1] + binary_secret_msg[data_index] 00011111
binary_secret_msg[data_index] 1
r 10101000
r[:-1] + binary_secret_msg[data_index] 10101001
binary_secret_msg[data_index] 0
r 11111111
r[:-1] + binary_secret_msg[data

In [44]:
Steganography()

 Steganography 
 1. Veri sakla 
 2. Saklananveriyi oku 
 Yukarıdaki seçeneklerden birini seçiniz: 2

Veri deşifreleniyor....
Veri saklanan resmin adını giriniz (örnek: gizli_!_test.png) :gizli_!_test.png
Resme gizlenen metin: 


decoded_data s
decoded_data se
decoded_data sel
decoded_data sela
decoded_data selam
decoded_data selam 
decoded_data selam n
decoded_data selam na
decoded_data selam nab
decoded_data selam nabe
decoded_data selam naber
decoded_data selam naber#
decoded_data selam naber##
decoded_data selam naber###
decoded_data selam naber####
decoded_data selam naber#####
Deşifrelenmiş verinin sonucu selam naber
